# Symbolic Solutions

Generates a table showing the symbolic models forms and solution rates

In [1]:
import pandas as pd
# import sympy 
from sympy import latex
from sympy.parsing.sympy_parser import parse_expr
df_results = pd.read_feather('results_sym_data_solns.feather')
df_results

,dataset,algorithm,random_state,process_time,training time (s),target_noise,feature_noise,true_model,model_size,symbolic_model,...,symbolic_error,symbolic_fraction,symbolic_error_is_zero,symbolic_error_is_constant,symbolic_fraction_is_constant,sympy_exception,training time (hr),r2_zero_test,data_group,symbolic_solution
0,feynman_II_37_1,SBP-GP,14423,28805.481642,28888.563637,0.000,0.0,B*mom*(chi + 1),399,7.573053+-3.186999*((2.389547+((-0.371850*(x0*...,...,-0.163*mom*cos(1) + 0.088*mom + 0.00806*cos(1)...,(1.0*B*chi*mom + 1.0*B*mom - 0.088*mom + 0.163...,False,False,False,None,8.024601,1.000000,Feynman,False
1,feynman_II_37_1,AFP_FE,4426,28818.569322,28897.310605,0.001,0.0,B*mom*(chi + 1),59,(sqrt(|((log((exp((x_0*x_2))+(sin(exp((x_0*x_0...,...,B*(chi*mom - log(exp(chi*mom) + sin(exp(mom**2...,(mom + log(exp(chi*mom) + sin(exp(mom**2))*cos...,False,False,False,None,8.027031,0.999993,Feynman,False
2,feynman_II_37_1,gplearn,29910,5848.466219,5851.717551,0.000,0.0,B*mom*(chi + 1),134,"add(add(add(add(add(sqrt(X0), cos(add(sub(X2, ...",...,-0.58*B*chi + B*mom - B + 2*chi - sqrt(mom) - ...,(B*chi*(mom + 0.58) + B - 2*chi + sqrt(mom) + ...,False,False,False,None,1.625477,0.998965,Feynman,False
3,feynman_II_37_1,SBP-GP,29910,28972.682290,28987.948357,0.001,0.0,B*mom*(chi + 1),839,43.103872+12.141448*(((0.999478*((1.001714*((1...,...,None,None,False,False,False,None,8.052208,0.997673,Feynman,False
4,feynman_II_37_1,FEAT,860,431.861356,442.143189,0.001,0.0,B*mom*(chi + 1),52,35.686+14.04*x_1+13.58*x_0+10.34*x_2+6.69*(0.0...,...,0.977*B*chi*mom - 0.194*B*chi + 0.838*B*mom - ...,(0.023*B*chi*mom + 0.236*B*(0.822*chi + 0.685*...,False,False,False,None,0.122818,0.999816,Feynman,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44460,feynman_test_1,EPLEX,16850,10370.496663,10400.154607,0.001,0.0,Z_1**2*Z_2**2*alpha**2*c**2*hbar**2/(16*E_n**2...,64,((((((((((x_3-sin(cos(x_1)))+sin(x_1))*0.83)^2...,...,0.322 - (Z_1**2*alpha**2*(c + sin(log(hbar)) -...,16*E_n**2*(-0.322 + (Z_1**2*alpha**2*(c + sin(...,False,False,False,None,2.888932,0.991547,Feynman,False
44461,feynman_test_1,SBP-GP,860,28854.281546,28910.303690,0.000,0.0,Z_1**2*Z_2**2*alpha**2*c**2*hbar**2/(16*E_n**2...,416,2.134858+4.038884*((cos((x5aq((sin(x2)+((plog(...,...,None,None,False,False,False,None,8.030640,0.978471,Feynman,False
44462,feynman_test_1,Operon,4426,3048.279373,3049.896474,0.001,0.0,Z_1**2*Z_2**2*alpha**2*c**2*hbar**2/(16*E_n**2...,88,((-0.002) + (1.000 * (((((((7.363 * X5) * (0.8...,...,0.00201 + Z_1**2*Z_2**2*alpha**2*c**2*hbar**2/...,16*E_n**2*(-0.00201 - 0.422*(0.014*Z_2 + alpha...,False,False,False,None,0.847193,0.999941,Feynman,False
44463,feynman_test_1,FFX,860,11.951047,12.059105,0.010,0.0,Z_1**2*Z_2**2*alpha**2*c**2*hbar**2/(16*E_n**2...,539,"-20.2 + 204*max(0,X2-1.73) * max(0,1.40-X6) - ...",...,-0.427*Z_1**2 - 9.530029296875*Z_1*Z_2 + 23.0*...,16*E_n**2*(0.427*Z_1**2 + 9.53*Z_1*Z_2 - 23.0*...,False,False,False,None,0.003350,0.750759,Feynman,False


In [ ]:
import pdb
from symbolic_utils import round_floats, complexity
from tqdm import tqdm

df_solns = df_results.loc[df_results.dataset.str.contains('strogatz')]
for (dataset,noise), dfg in tqdm(df_solns.groupby(['dataset','target_noise'])):
    tbl = []
#     print(80*'=')
    tbl.append('\\begin{tabularx}{\\textwidth}{l Z}') 
#     tbl.append('\t&\t'.join(['Name','Ground Truth'])) 
#     tbl.append('\\midrule') 
#     print(dataset,noise)
#     print(dfg['true_model'].dropna())
    tm = dfg['true_model'].unique()[0]
    model = round_floats(parse_expr(tm, evaluate=False))
    prefix = '\dot{x} = ' if '1' in dataset else '\dot{y} = '
    tbl.append('\t&\t'.join([dataset, '$'+prefix+latex(model)+'$']))
#     print('True Model:',latex(model))
    dfg_solns = dfg.loc[dfg.symbolic_solution==True]
    tbl.append('\\midrule') 
    tbl.append('Method&\tEstimated Model')
    soln_algs = []
    if len(dfg_solns)>0:
#         print('Solutions',60*'=')
#         tbl.append('\\multicolumn{2}{c}{Solutions}')
        for alg, dfga in dfg_solns.groupby('algorithm'):
            sm = dfga['simplified_symbolic_model'].values[0] 
            model = round_floats(parse_expr(sm, evaluate=False))
#             if str(model) == '0':
#                 pdb.set_trace()
#             print('  ',alg,'model:',model)
            mstr = '$'+prefix+latex(model)+'$'
            tbl.append('\t&\t'.join(['\\textbf{'+alg+'}', mstr]))
            soln_algs.append(alg)
        tbl.append('\\midrule') 
        
    df_notsolns = dfg.loc[dfg.symbolic_solution==False]
    df_notsolns = df_notsolns.loc[~df_notsolns.algorithm.isin(soln_algs)]
#     print('Non-solutions',56*'=')
#         tbl.append('Method ($R^2$)\t&\tNon-solutions')
    for alg, dfga in df_notsolns.groupby('algorithm'):
        guesses = dfga['simplified_symbolic_model'].values
        accs = dfga['r2_test'].values
        if len(guesses) == 0:
            continue
#             print(guesses) 
#             acc = dfga['r2_test'].values[0]
        guess_model = None
        best_acc = None
        for i,(g,acc) in enumerate(zip(guesses,accs)):
            if g:
                gm = round_floats(parse_expr(g, evaluate=False))
                if complexity(gm)>40 or 'NaN' in latex(gm):
                    continue
                else:
                    if best_acc==None or acc > best_acc:
#                         print('adding gm w complexity',complexity(gm))
                        guess_model = latex(gm)
                        best_acc = acc


        if guess_model:
            if str(guess_model)=='0': 
                continue
            mstr = '$'+prefix+guess_model+'$'
            tbl.append('\t&\t'.join([alg, mstr]))

    #         for g in guess:
    #             print('\t',g[:80],'...' if len(g)>80 else '')
    #             print('\t',g)
    tbl.append('\\bottomrule') 
    tbl.append('\\end{tabularx}')
    tbl = [t+'\t\\\\' if all([s not in t for s in ['tabular','rule','hline']]) else t for t in tbl ]
#     tbl = ['\\begin{table}\n\\tiny\n\\centering\n']+tbl+['\\end{table}\n']
    filename = '../../paper/tables/results_sym_data/'+'_'.join([dataset,str(noise)])+'_sym_models.tex'
    with open(filename, 'w') as f:
        f.write('\n'.join(tbl).replace('_','\\_'))


  0%|          | 0/42 [00:00<?, ?it/s]

adding gm w complexity 26
adding gm w complexity 14
adding gm w complexity 38
adding gm w complexity 23
adding gm w complexity 33
adding gm w complexity 33
adding gm w complexity 3
adding gm w complexity 25
adding gm w complexity 32
adding gm w complexity 29
adding gm w complexity 35
adding gm w complexity 25
adding gm w complexity 31
adding gm w complexity 11
adding gm w complexity 23
adding gm w complexity 30
adding gm w complexity 1


  2%|▏         | 1/42 [00:07<05:27,  7.99s/it]

adding gm w complexity 34
adding gm w complexity 20
adding gm w complexity 20
adding gm w complexity 38
adding gm w complexity 39
adding gm w complexity 38
adding gm w complexity 5
adding gm w complexity 34
adding gm w complexity 32
adding gm w complexity 14
adding gm w complexity 22
adding gm w complexity 34
adding gm w complexity 32
adding gm w complexity 29
adding gm w complexity 39
adding gm w complexity 35
adding gm w complexity 11
adding gm w complexity 11
adding gm w complexity 11
adding gm w complexity 11
adding gm w complexity 25


  5%|▍         | 2/42 [00:17<05:32,  8.30s/it]

...

adding gm w complexity 10
adding gm w complexity 4
adding gm w complexity 7
adding gm w complexity 25
adding gm w complexity 25
adding gm w complexity 36
adding gm w complexity 39
adding gm w complexity 1
adding gm w complexity 9
adding gm w complexity 40
adding gm w complexity 26
adding gm w complexity 18
adding gm w complexity 5
adding gm w complexity 30
adding gm w complexity 28
adding gm w complexity 34
adding gm w complexity 36
adding gm w complexity 39
adding gm w complexity 12
adding gm w complexity 11
adding gm w complexity 9


 21%|██▏       | 9/42 [03:35<12:37, 22.97s/it]

adding gm w complexity 10
adding gm w complexity 20
adding gm w complexity 38
adding gm w complexity 31
adding gm w complexity 31
adding gm w complexity 39
adding gm w complexity 11
adding gm w complexity 11
adding gm w complexity 11
adding gm w complexity 11
adding gm w complexity 20
adding gm w complexity 28
adding gm w complexity 38
adding gm w complexity 25
adding gm w complexity 25
adding gm w complexity 33
adding gm w complexity 13
adding gm w complexity 14
adding gm w complexity 13
adding gm w complexity 13
adding gm w complexity 24


 24%|██▍       | 10/42 [03:43<09:54, 18.59s/it]

adding gm w complexity 26
adding gm w complexity 7
adding gm w complexity 26
adding gm w complexity 23
adding gm w complexity 38
adding gm w complexity 14
adding gm w complexity 25
adding gm w complexity 39
adding gm w complexity 33
adding gm w complexity 25
adding gm w complexity 11
adding gm w complexity 29
adding gm w complexity 23
adding gm w complexity 29
adding gm w complexity 14
adding gm w complexity 13
adding gm w complexity 13


In [ ]:
df_solns = df_results.loc[df_results.dataset.str.contains('strogatz_bacres1')]
df_solns = df_solns.loc[df_solns.algorithm=='ITEA']
df_solns = df_solns.loc[df_solns.target_noise==0.0]
for (tm, sm), dfg in df_solns.groupby(['true_model','simplified_symbolic_model']):
    print(tm)
    print('\t',sm)

# strogatz

In [ ]:
df_strogatz = df_results.loc[df_results.dataset.str.contains('strogatz')]
tbl = ['\\begin{tabularx}{\\textwidth}{lr} \\\\'
       'Dataset 
      ]
for dataset, dfg in df_strogatz.groupby('dataset'):
    tbl.append(
        ''
    )
    

In [ ]:
df_sum = pd.read_csv('symbolic_dataset_results_sum.csv.gz') 
df_strogatz_sum = df_sum.loc[df_sum.dataset.str.contains('strogatz'),:] 
df_feynman_sum = df_sum.loc[df_sum.dataset.str.contains('feynman'),:] 
strogatz_solns = df_strogatz_sum.set_index(['dataset','target_noise','algorithm'])['symbolic_solution_rate_(%)'].unstack().round(0)
feynman_solns = df_feynman_sum.set_index(['dataset','target_noise','algorithm'])['symbolic_solution_rate_(%)'].unstack().round(0)
strogatz_solns.to_latex('../../paper/tables/strogatz_soln_pct.tex')

In [ ]:
df_sum.loc[df_sum.dataset.str.contains('strogatz'),:]